In [3]:
import csv
import time
import openpyxl
import math
import os
import pandas as pd
os.chdir(r'C:\Users\op1\Desktop\Pyfile\上手風控\data')
class productInfo:
    equityscg = {'USD':0,'JPY':0,'SGD':0,'EUR':0,'HKD':0}
    marginscg = {'USD':0,'JPY':0,'SGD':0,'EUR':0,'HKD':0}
    def __init__(self, contract=None, month=None,exchange=None,
                 Buy_oi=0,Sell_oi=0,tick=[0,0],margin=0,setP=0,currency=None,
                TDate=None,Broker='SCG'):
        
        self.contract = contract
        self.month = month
        self.exchange = exchange
        self.Buy_oi = Buy_oi
        self.Sell_oi = Sell_oi
        self.tick = tick
        self.margin = margin
        self.setP = setP
        self.currency = currency
        self.TDate = TDate
        self.Broker = Broker
        
    def change(self):
        with open('商品轉換.csv','rt',encoding = 'utf-8') as fphi:
            phifile = csv.reader(fphi)
            check_product = 0
            for i in phifile:
                if self.contract == i[0]:
                    self.contract = i[1]
                    self.exchange = i[2]
                    self.currency = i[3]
                    #self.tick[0] = i[4]
                    #self.tick[1]= i[5]
                    check_product += 1  
            if check_product == 0 :
                print('請確認',self.contract,'找不到對照商品')
                
    def pimargin(self):
        with open('//10.36.205.170/交易結算部/國外/交易量+未平倉/' +date[5:]+'/保證金.TXT','r') as mar:
            marR = mar.readlines()
            for i in marR:
                if len(i.split())==11 and i.split()[2]==self.contract:
                    self.margin = int(i.split()[6].replace('.00','').replace(',',''))

    def piINFO(self):
        return([
            self.Broker ,
            self.contract,
            self.month,
            self.exchange, 
            self.Buy_oi,
            self.Sell_oi,
            #self.tick , 
            self.margin,
            abs((self.Buy_oi - self.Sell_oi) * self.margin),
            self.setP,
            self.currency
        ])

   
 #--------------------------------------------------上手資料----------------------------------------------- 

fileList = os.listdir()
fileList = [i for i in os.listdir() if i[-3:] == 'txt' and 'STMT' in i]

for i in fileList:
    scg= open(i, "r")
    R = scg.readlines()
    scg.close()
    productInfo.equityifp = {'USD':0,'JPY':0,'SGD':0,'EUR':0,'HKD':0}
    productInfo.marginifp = {'USD':0,'JPY':0,'SGD':0,'EUR':0,'HKD':0}

    scgOI=[]

    exchange = ['CBOE','CBOT','CME','COMEX','NYBOT','NYMEX','SGX']

    for i,p in enumerate(R):                             #切割資料成list
        if ('O  P  E  N      P  O  S  I  T  I  O  N  S') in p:
            k = i
            

    SCG = productInfo()

    SCG.TDate = (' '.join(R[4].split())).replace(',','')
    date0 = time.strptime(SCG.TDate, '%b %d %Y')
    date1 = time.strftime("%Y/%m/%d",date0)   #資料日期
    date = time.strftime("%Y-%m%d",date0)     #sheets名稱資料日期


    for i in range(k,len(R)):
        if R[i].find('CLOSE') > 0 :
            source1 = R[i].split()    
            source0 = R[i-1].split()

            SCG.Buy_oi = 0
            SCG.Sell_oi = 0
            if R[i].count('*') == 2 :
                SCG.Buy_oi = int(source1[0].replace('*','').replace(',',''))
                SCG.Sell_oi = int('-' + source1[1].replace('*','').replace(',',''))
            elif R[i].find('*', 0, 40) > 0 :   
                SCG.Buy_oi = int(source1[0].replace('*','').replace(',',''))
            elif R[i].find('*', 40, 60) > 0 :
                SCG.Sell_oi = int('-' + source1[0].replace('*','').replace(',',''))

            if source0.count('PUT') == 0 and source0.count('CALL') == 0:
                SCG.month = source0[2] + source0[3]
                if len(source0) == 8:
                    SCG.contract = source0[4]             
                elif len(source0) == 9:
                    SCG.contract = source0[4] + '-' + source0[5]
                elif len(source0) == 10: 
                    SCG.contract = source0[4] + '-' + source0[5] + '-' + source0[6]
                elif len(source0) == 12: 
                    SCG.contract = source0[7]
                    SCG.month = source0[4] + source0[5]
                else:
                    print('多出商品 : ', source0)

                SCG.change()        
                SCG.pimargin()
                scgOI.append(SCG.piINFO())

        if  R[i].find('INITIAL MARGIN REQUIREMENT') >= 0 and len(R[i]) > 60 :
            source1 = R[i].split()
            productInfo.marginscg['USD'] = int(source1[3][:-3].replace(',',''))
            try:
                productInfo.marginscg['JPY'] = int(source1[4][:-3].replace(',','')) + int(source1[5][:-3].replace(',',''))
            except ValueError:
                productInfo.marginscg['JPY'] = 0
            try:        
                productInfo.marginscg['SGD'] = int(source1[6][:-3].replace(',',''))
            except ValueError:
                productInfo.marginscg['SGD'] = 0


        if  R[i].find('TOTAL EQUITY') >= 0 and len(R[i]) > 60 :
            source1 = R[i].split()
            productInfo.equityscg['USD'] = int(source1[2][:-3].replace(',',''))
            try:
                productInfo.equityscg['JPY'] = int(source1[3][:-3].replace(',','')) + int(source1[4][:-3].replace(',',''))
            except ValueError:
                productInfo.equityscg['JPY'] = 0
            try:
                productInfo.equityscg['SGD'] = int(source1[5][:-3].replace(',',''))
            except ValueError:
                productInfo.equityscg['SGD'] = 0
    
    txt1 = open(f'//10.36.205.170/交易結算部/0.暫存/壓力測試/目標/{date}結算價.TXT', "r")
    date_margine = time.strftime("%Y/%m/%d",date0)
    r1 = txt1.readlines()
    k = [i.split() for i in r1 if i.split()[0] == date_margine]
    if k == []: print('結算價檔日期錯誤')
    E1 = {'JAN':'01','FEB':'02','MAR':'03','APR':'04','MAY':'05','JUN':'06',
         'JUL':'07','AUG':'08','SEP':'09','OCT':'10','NOV':'11','DEC':'12',}
    
    
    
    t1 = pd.read_csv('商品轉換.csv',usecols=['商品名', '分母值'])  
    t1.drop_duplicates(inplace=True)
    t1.set_index(['商品名'], inplace=True)
    t1 = t1[t1['分母值']>1]
    for i in scgOI:
        for j in k:
            a = '20' + i[2][3:] + '/' + E1[i[2][:3]]

            b = i[1]
            if a in j and b in j:
                try:
                    d1 = int(t1.loc[j[2], '分母值'])  # 取分母值 : 4
                    d1_len = len(str(d1))             # 分母值幾位數 
                    settle = j[j.index("期貨")+2]     # 結算價
                    re_settle = settle.split('.')     # 1700.200000000000 
                    n1 = int(re_settle[1][0:d1_len]) / d1 # .200000000 取 .2 取 2 除 4 得 0.5
                    n1 = str(n1).split('.')[1]          # 0.5 取 5
                    re_settle[1] = n1                  # ['1,700', '5']
                    i[8] = '.'.join(re_settle)        # ['1,700.5']
                except KeyError:
                    i[8] = j[j.index("期貨")+2]
    txt1.close()    
    
    curL = ['USD','JPY','EUR']
    for cur in curL:
        usdS = []
        usdN = []
        for i in scgOI:
            if i[-1] ==cur:
                usdS.append(i[-3])
                usdN.append(i[1]+i[2])

        for i in range(len(scgOI)):
            if cur in scgOI[i]:
                marPPS = scgOI[i][-3]/sum(usdS)
                scgOI[i].append(marPPS)
                scgOI[i].append(int(scgOI[i][-1]*productInfo.marginscg[cur]))

     #--------------------------------------------------填回資料----------------------------------------------- 



    wb = openpyxl.load_workbook('RiskControl.xlsx')

    title=['上手','商品','月份','交易所','買方口數',
           '賣方口數','保證金','加總','結算價','幣別','占比','調整後保證金']

    scgEqtAndMar = [
                    date1, SCG.Broker,
                    productInfo.equityscg['USD'],productInfo.equityscg['EUR'],
                    productInfo.equityscg['JPY'],productInfo.equityscg['HKD'],
                    productInfo.equityscg['SGD'],
                    productInfo.marginscg['USD'],productInfo.marginscg['EUR'],
                    productInfo.marginscg['JPY'],productInfo.marginscg['HKD'],
                    productInfo.marginscg['SGD'],                
                   ]

    ##....填回每日保證金

    try:                      #判斷sheet是否存在
        ws = wb[date]
    except KeyError:
        wb.create_sheet(date)
        ws = wb[date]
        ws.append(title)

    t = 0                     #判斷資料是否存在
    for i in ws['A']:
        if 'SCG' in i.value:
            t=1
            print(date+ '已有資料')
            break       
    if t == 0 :               #資料填入
        for row in range(len(scgOI)):
            ws.append(scgOI[row])

    ##....填回歷史資料

    ws = wb["HistoryData"]

    t = 0
    for i in ws['A']:         #判斷資料是否存在
        if date1 in i.value:
            if 'SCG' in ws.cell(row = i.row , column = 2).value:
                t=1
                break        
    if t == 0 :               #資料填入
        ws.append(scgEqtAndMar)        
    wb.save('RiskControl.xlsx')
    print(SCG.Broker+date+' OK')



SCG2021-0402 OK
SCG2021-0405 OK
SCG2021-0406 OK
SCG2021-0407 OK
SCG2021-0409 OK
SCG2021-0412 OK
SCG2021-0413 OK
SCG2021-0414 OK
SCG2021-0415 OK
SCG2021-0416 OK
SCG2021-0419 OK
SCG2021-0420 OK
SCG2021-0421 OK
SCG2021-0422 OK
SCG2021-0423 OK
SCG2021-0426 OK
SCG2021-0427 OK
SCG2021-0428 OK
SCG2021-0429 OK
SCG2021-0430 OK
SCG2021-0503 OK
SCG2021-0504 OK
SCG2021-0505 OK
SCG2021-0507 OK
SCG2021-0510 OK
SCG2021-0511 OK
SCG2021-0512 OK
SCG2021-0514 OK
SCG2021-0517 OK
SCG2021-0518 OK
SCG2021-0519 OK
SCG2021-0520 OK
SCG2021-0521 OK
SCG2021-0524 OK
SCG2021-0525 OK
SCG2021-0526 OK
SCG2021-0527 OK
SCG2021-0528 OK
